In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers

In [106]:
# turn list of lists of equivalent braids with corresponding answers into trainable data

def parse_equivalent_braids(allBraids, answers, maxCrossings):

    x_data = []
    y_data = []

    for equivalentBraids, answer in zip(allBraids, answers):
        for braid in tf.keras.preprocessing.sequence.pad_sequences(
            equivalentBraids, padding="post", maxlen=maxCrossings):
            x_data.append(braid)
            y_data.append(answer)
        
    return np.asarray(x_data), np.asarray(y_data)

In [107]:
# read pre-parsed knot data with braid words and equivalent braids

knot_data = pd.read_csv('../data/knot_volume.csv')
knot_data["Equivalent Braids"] = knot_data["Equivalent Braids"].apply(eval)

In [108]:
eq = knot_data["Equivalent Braids"]

maxCrossings = max([max([len(braid) for braid in braids]) for braids in eq])
maxStrands = max([max([max([abs(b) for b in braid]) for braid in braids]) for braids in eq])

In [109]:
# split dataset for training

train_split = .8

train_data = knot_data.sample(frac=train_split)
test_data = knot_data.drop(train_data.index)

x_train, y_train = parse_equivalent_braids(train_data["Equivalent Braids"],
                    train_data["Volume"], maxCrossings)

x_train = tf.keras.utils.to_categorical(x_train, num_classes=maxStrands*2+1)

x_test, y_test = parse_equivalent_braids(test_data["Equivalent Braids"],
                    test_data["Volume"], maxCrossings)

x_test = tf.keras.utils.to_categorical(x_test, num_classes=maxStrands*2+1)


In [110]:
# build model from functional api

inputs = keras.Input(shape=(maxCrossings, maxStrands*2+1))

x = layers.Flatten()(inputs)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1)(x)

In [111]:
model = keras.Model(inputs=inputs, outputs=outputs, 
    name="quasipositivity_functional")

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_squared_error"])

In [112]:
model.summary()

Model: "quasipositivity_functional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 19, 13)]          0         
                                                                 
 flatten_6 (Flatten)         (None, 247)               0         
                                                                 
 dense_18 (Dense)            (None, 64)                15872     
                                                                 
 dense_19 (Dense)            (None, 64)                4160      
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                                 
Total params: 20,097
Trainable params: 20,097
Non-trainable params: 0
_________________________________________________________________


In [113]:
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=5
)

Epoch 1/5
1189/1189 [==============================] - 1s 539us/step - loss: 16.1262 - mean_squared_error: 16.1262
Epoch 2/5
1189/1189 [==============================] - 1s 524us/step - loss: 9.2984 - mean_squared_error: 9.2984
Epoch 3/5
1189/1189 [==============================] - 1s 523us/step - loss: 8.1201 - mean_squared_error: 8.1201
Epoch 4/5
1189/1189 [==============================] - 1s 517us/step - loss: 7.6488 - mean_squared_error: 7.6488
Epoch 5/5
1189/1189 [==============================] - 1s 513us/step - loss: 7.3550 - mean_squared_error: 7.3550


In [114]:
model.evaluate(x_test, y_test)

595/595 [==============================] - 0s 332us/step - loss: 8.3392 - mean_squared_error: 8.3392


[8.33924388885498, 8.33924388885498]

In [116]:
def predict_volume(braid_word):
    padded_braid = np.pad(braid_word, (0, maxCrossings - len(braid_word)))
    encoded_braid = tf.keras.utils.to_categorical(padded_braid, num_classes=maxStrands*2+1)
    return np.asarray(model(np.array([encoded_braid])))[0][0]

In [117]:

predict_volume([1, -2, 1, 2, 3, -2, 3, -4, 2, -4, -3, 2, -3, 4])

12.821954